# Baseline Backtest (Original Version)
This notebook demonstrates the **raw IV-spread straddle arbitrage** without Kalman filtering.
We compute rolling mean/std, generate entry/exit signals on z-score, and approximate PnL via
mean-reversion of the spread deviation.


In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
df = pd.read_csv('../data/iv_spread_sample.csv', parse_dates=['date']).set_index('date')
s = df['iv_spread']
print(df.head()); print(f"Data points: {len(df)}")


In [ ]:

def annualize_return(daily_returns, trading_days=252):
    daily_returns = daily_returns.fillna(0.0)
    cum = (1 + daily_returns).prod()
    years = len(daily_returns) / trading_days
    return cum**(1/years) - 1

def sharpe_ratio(daily_returns, trading_days=252):
    daily_returns = daily_returns.fillna(0.0)
    mu = daily_returns.mean()
    sigma = daily_returns.std(ddof=1)
    return 0.0 if sigma==0 else (mu * trading_days) / (sigma * np.sqrt(trading_days))

def max_drawdown(equity):
    eq = equity.fillna(method="ffill")
    peak = eq.cummax()
    dd = eq/peak - 1.0
    return float(dd.min())


In [ ]:

def baseline_backtest(series, window=126, entry_z=2.0, exit_z=0.5, max_hold=10,
                      trade_cost_bp=8, slippage_bp=8, pnl_scale=0.003):
    s = series.copy()
    roll_mu = s.rolling(window).mean()
    roll_sd = s.rolling(window).std().replace(0, np.nan)
    z = (s - roll_mu) / roll_sd

    position = 0; hold = 0
    daily_pnl = pd.Series(0.0, index=s.index)
    equity = pd.Series(1.0, index=s.index)
    prev_dev = s.iloc[0] - (roll_mu.iloc[0] if not np.isnan(roll_mu.iloc[0]) else 0.0)

    for i in range(1, len(s)):
        if np.isnan(roll_sd.iloc[i]) or np.isnan(roll_mu.iloc[i]):
            equity.iloc[i] = equity.iloc[i-1]
            prev_dev = s.iloc[i] - (roll_mu.iloc[i] if not np.isnan(roll_mu.iloc[i]) else 0.0)
            continue
        dev = s.iloc[i] - roll_mu.iloc[i]
        dev_prev = prev_dev

        if position == 0:
            if z.iloc[i] > entry_z:
                position = +1; hold = 0
                daily_pnl.iloc[i] -= (trade_cost_bp + slippage_bp)/1e4
            elif z.iloc[i] < -entry_z:
                position = -1; hold = 0
                daily_pnl.iloc[i] -= (trade_cost_bp + slippage_bp)/1e4
        else:
            hold += 1
            if abs(z.iloc[i]) <= exit_z or hold >= max_hold:
                position = 0; hold = 0
                daily_pnl.iloc[i] -= (trade_cost_bp + slippage_bp)/1e4

        if position != 0:
            daily_pnl.iloc[i] += position * (abs(dev_prev) - abs(dev)) * pnl_scale

        equity.iloc[i] = equity.iloc[i-1] * (1 + daily_pnl.iloc[i])
        prev_dev = dev

    return {"equity": equity, "daily_ret": daily_pnl, "z": z}


In [ ]:

res = baseline_backtest(s)
print(f"Annualized Return: {annualize_return(res['daily_ret']):.2%}")
print(f"Sharpe Ratio     : {sharpe_ratio(res['daily_ret']):.2f}")
print(f"Max Drawdown     : {max_drawdown(res['equity']):.2%}")


In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(9,4))
plt.plot(res["equity"].index, res["equity"].values)
plt.title("Baseline Strategy Equity Curve (Synthetic)")
plt.xlabel("Date"); plt.ylabel("Equity"); plt.tight_layout()
plt.show()

from pathlib import Path
Path('../results').mkdir(exist_ok=True, parents=True)
plt.figure(figsize=(9,4))
plt.plot(res["equity"].index, res["equity"].values)
plt.title("Baseline Strategy Equity Curve (Synthetic)")
plt.xlabel("Date"); plt.ylabel("Equity"); plt.tight_layout()
plt.savefig('../results/baseline_equity.png', dpi=150)
plt.close()
